<a href="https://colab.research.google.com/github/tienhuynh96/poem-generator/blob/main/%5BDemo%5D_Poem_Generation_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Install and import libraries**

In [ ]:
!pip install -qq datasets==2.16.1 evaluate==0.4.1 transformers[sentencepiece]==4.35.2
!pip install -qq accelerate==0.26.1
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 74.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 

In [ ]:
import os
import math
import torch
import pandas as pd
# Import GPT tokenizer and LMHeadModel
from transformers import GPT2Tokenizer, GPT2LMHeadModel
# DataCollatorForLanguageModeling for setting model is generator next token model
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login
from datasets import Dataset

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
notebook_login()

## **2. Download and load poem dataset**

In [ ]:
# https://drive.google.com/file/d/1KfrBAycsgQBt1mtEbzJh5pSYL8YIk0Tc/view?usp=sharing
!gdown --id 1KfrBAycsgQBt1mtEbzJh5pSYL8YIk0Tc
!unzip poem_dataset_final.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1KfrBAycsgQBt1mtEbzJh5pSYL8YIk0Tc
To: /content/poem_dataset_final.zip
100% 37.6k/37.6k [00:00<00:00, 93.9MB/s]
Archive:  poem_dataset_final.zip
  inflating: poem_final.csv          


In [ ]:
DATASET_PATH = 'poem_final.csv'
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,3,"“Đến, nhiều nơi để đến”","Đến, nhiều nơi để đến\nVề, trở lại với mình\nC...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Đừng bao giờ dại dột”,Đừng bao giờ dại dột\nĐem chuyện riêng của mìn...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
185,95,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
186,96,Áng dương không biết sầu,Áng dương không biết sầu\nNằm mãi ở trên cao\n...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
187,97,Anh,Cây bút gẫy trong tay\nCặn mực khô đáy lọ\nÁnh...,19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
188,98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [ ]:
print(df['content'][0])

Cái làm ta hạnh phúc
Thực ra cũng chẳng nhiều
Chỉ cần có ai đó
Để ta thầm thương yêu

Rồi thêm chút công việc
Cho ta làm hàng ngày
Cuối cùng, chút mơ mộng
Để đưa ta lên mây


In [ ]:
## Split_content function to separate paragraph and line of each sample poem
def split_content(content):
    samples = []

    poem_parts = content.split('\n\n')
    for poem_part in poem_parts:
        poem_in_lines = poem_part.split('\n')
        if len(poem_in_lines) == 4:
            samples.append(poem_in_lines)

    return samples

df['content'] = df['content'].apply(lambda x: split_content(x))
df['content'][0]

[['Cái làm ta hạnh phúc',
  'Thực ra cũng chẳng nhiều',
  'Chỉ cần có ai đó',
  'Để ta thầm thương yêu'],
 ['Rồi thêm chút công việc',
  'Cho ta làm hàng ngày',
  'Cuối cùng, chút mơ mộng',
  'Để đưa ta lên mây']]

In [ ]:
# Explode content row in paragrap of 4 lines
df_exploded = df.explode('content')
# Reset index
df_exploded.reset_index(drop=True, inplace=True)
# Remove NAN
df_exploded = df_exploded.dropna(subset=['content'])
df_exploded

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,"[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhiề...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,“Cái làm ta hạnh phúc”,"[Rồi thêm chút công việc, Cho ta làm hàng ngày...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,“Chiều vừa xốp trên tay”,"[Chiều vừa xốp trên tay, Chợt nghe thoáng ong ...","Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,1,“Chiều vừa xốp trên tay”,"[Ớt đỏ sao cứ đỏ, Táo chín cho thật vàng, Em đ...","Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
4,2,“Dưới giàn hoa thiên lý...”,"[Dưới giàn hoa thiên lý, Một mình anh đang ngồ...","Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
...,...,...,...,...,...
481,98,Anh biết,"[Đất hiến dâng tuổi trẻ, Trời buông thả tuổi g...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
482,99,Anh bốn mùa lập đông,"[Buồn, thì buồn thiệt đó, Em à, có biết không,...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
483,99,Anh bốn mùa lập đông,"[Vâng, phiến buồn còn đó, Em nào biết phải khô...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
484,99,Anh bốn mùa lập đông,"[Ừ, nỗi buồn vậy đó, Em đừng vội bâng khuâng, ...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [ ]:
df_exploded['content'][0]

['Cái làm ta hạnh phúc',
 'Thực ra cũng chẳng nhiều',
 'Chỉ cần có ai đó',
 'Để ta thầm thương yêu']

In [ ]:
# Join each sample and add token \n
df_exploded['content'] = df_exploded['content'].apply(lambda x: '\n'.join(x))
df_exploded

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,“Cái làm ta hạnh phúc”,Rồi thêm chút công việc\nCho ta làm hàng ngày\...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,1,“Chiều vừa xốp trên tay”,Ớt đỏ sao cứ đỏ\nTáo chín cho thật vàng\nEm đẹ...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
4,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
...,...,...,...,...,...
481,98,Anh biết,Đất hiến dâng tuổi trẻ\nTrời buông thả tuổi gi...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
482,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
483,99,Anh bốn mùa lập đông,"Vâng, phiến buồn còn đó\nEm nào biết phải khôn...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
484,99,Anh bốn mùa lập đông,"Ừ, nỗi buồn vậy đó\nEm đừng vội bâng khuâng\nV...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [ ]:
# Put dataframe into Dataset (hugging face)
poem_dataset = Dataset.from_pandas(df_exploded)
poem_dataset


Dataset({
    features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
    num_rows: 441
})

In [ ]:
# split to train and test dataset
TEST_SIZE = 0.1
poem_dataset = poem_dataset.train_test_split(test_size=TEST_SIZE)
poem_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 396
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 45
    })
})

## **3. Preprocess dataset**

In [ ]:
# Set model name
MODEL_NAME = 'danghuy1999/gpt2-viwiki'
# Set tokenizer from pre-trained
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/773k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/431k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

In [ ]:
print(tokenizer.pad_token)
print(tokenizer.eos_token)

None
<|endoftext|>


In [ ]:
# Set pad_token = eos token, to indicate when padding is the end of sequence
tokenizer.pad_token = tokenizer.eos_token
# Set max sequence length
MAX_SEQ_LEN = 100

# Create preprocess_function
def preprocess_function(row):
  return tokenizer(
      row['content'], # Process each row
      max_length=MAX_SEQ_LEN,   # the tokenized sequence has a maximum length of 100 tokens.
      padding='max_length', # Pads sequences shorter than 100 tokens to exactly 100 tokens.
      truncation=True       # Set truncation
  )

In [ ]:
tokenizer('xin chào các bạn', max_length=10, padding='max_length', truncation=True)

{'input_ids': [6504, 3939, 331, 1486, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]}

In [ ]:
preprocess_function(poem_dataset['train'][0])

{'input_ids': [4700, 2078, 742, 682, 2318, 199, 6796, 2078, 742, 682, 2189, 199, 26368, 2061, 2143, 1131, 1221, 199, 14661, 2078, 430, 1787, 2414, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [ ]:
# Applies the function to each element in the dataset.
tokenized_poem_dataset = poem_dataset.map(
    preprocess_function,
    batched=True, # Processes multiple rows at once
    num_proc=4,   # Uses 4 processes to speed up the preprocessing
    remove_columns=poem_dataset['train'].column_names, # Removes the original columns after preprocessing, keeping only the tokenized data
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/396 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/45 [00:00<?, ? examples/s]

In [ ]:
# Check key each column
tokenized_poem_dataset['train'][0].keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
# Set model for generate language model (for auto set up required suitable input)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
data_collator

DataCollatorForLanguageModeling(tokenizer=GPT2Tokenizer(name_or_path='danghuy1999/gpt2-viwiki', vocab_size=50257, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}, mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

## **4. Training**

In [ ]:
# Set model is GPT2
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [ ]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='gpt2_viet_poem_generation',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=True
)

In [ ]:
# Set up trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poem_dataset['train'],
    eval_dataset=tokenized_poem_dataset['test'],
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [ ]:
# Train model
trainer.train()

Step,Training Loss
500,5.818300


TrainOutput(global_step=500, training_loss=5.8182802734375, metrics={'train_runtime': 213.62, 'train_samples_per_second': 18.538, 'train_steps_per_second': 2.341, 'total_flos': 202093056000000.0, 'train_loss': 5.8182802734375, 'epoch': 10.0})

In [ ]:
# Push trained model to hub
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/tienhuynh/gpt2_viet_poem_generation/commit/fff0d2103761cceaedd89be6078dff23a67d9999', commit_message='Training complete', commit_description='', oid='fff0d2103761cceaedd89be6078dff23a67d9999', pr_url=None, pr_revision=None, pr_num=None)

## **5. Inference**

In [ ]:
# Inference way 1: use pipeline and trained model in huggingface
from transformers import pipeline

prompt = 'Con sông quê tôi đẹp\n'
# Create generator py pipeline
generator = pipeline('text-generation', model='tienhuynh/gpt2_viet_poem_generation')
results = generator(
    prompt,
    max_new_tokens=50,
    do_sample=True, # Set choose sample by true (if false is greedy decoding)
    top_k=50,       # Implements top-k sampling
    top_p=0.95,     #  Implements nucleus sampling (also known as top-p sampling)
    temperature=0.8,  # Controls the randomness of predictions by scaling the logits before applying softmax
    repetition_penalty=1.2  # Set repetition penalty
)
# results = [{'generated_text': "text result"}]
results = results[0]['generated_text']
print()
for line in results.split('\n'):
    print(line)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Con sông quê tôi đẹp
Chạn con cái, buồn bên trên cây cối
S ngọn núi không còn nữa nữa
Những người đi qua những cơn mưa
Khi những quan hệ có thể nhìn mình
Có nhiều cảnh rừng đẹp
Các cây tình yêu trời
Không thật


In [ ]:
# Inference way 2: use trained model
prompt = 'Học học nữa học mãi\n'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
# Use generate in model
outputs = model.generate(
    inputs,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2
)
# decpde outputs
results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
results = results[0]
print()
for line in results.split('\n'):
    print(line)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Học học nữa học mãi
Còn còn có những gì
Cho biết, ai không biết
Một người đã chết thì đang sống
Chỉ từ đâu trong làng?
Nhưng ai tôi?
Người khác! nhớ!
Dừng lại! buồn, đau khổ!
Để
